In [1]:
import os
import shutil
import numpy as np
import h5py
import matplotlib.pyplot as plt
plt.rc('font', size=12.0)

import tensorflow as tf
from keras import layers, mixed_precision, regularizers
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

2024-10-30 16:22:56.671099: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-30 16:22:56.680885: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-30 16:22:56.683893: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 16:22:56.691977: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model = ['nl']
label = ['NL']
x_names = ['a', 'alpha', 'param_S', 'param_L', '', 'trans1', '', 'trans2', '']
for k in range(len(model)):
    if model[k] == 'mp' or model[k] == 'np':
        x_names[4], x_names[6], x_names[8] = 'exp1', 'exp2', 'exp3'
    else:
        x_names[4], x_names[6], x_names[8] = 'csq1', 'csq2', 'csq3'
y_names = [f"R_{i}" for i in range(100)]

# Define file directory and load data
dir = '/home/anik/bamr/out/aff_inv/'
mchain = h5py.File(dir + 'nl_all', 'r')['markov_chain_0']

# Prepare X, Y, Z based on loaded data
x_ncols, y_ncols = len(x_names), len(y_names)
nrows, data = mchain['nlines'][0], mchain['data']
X, Y = np.zeros((x_ncols, nrows)), np.zeros((y_ncols, nrows))

for i in range(x_ncols):
    X[i] = data[x_names[i]]
for i in range(y_ncols):
    Y[i] = data[y_names[i]]
Z = np.array(data['R_max']).reshape(-1, 1)  # Shape Z as (nrows, 1)

# Transpose X and Y to shape (nrows, ncols)
X, Y = X.T, Y.T

# Initialize scalers
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_Y = MinMaxScaler(feature_range=(0, 1))  # Shared scaler for Y and Z

# Fit-transform X
x = scaler_X.fit_transform(X)

# Mask and scale non-zero values in Y
nz = Y != 0
y = np.copy(Y)  # Copy Y to keep zeros intact
y[nz] = scaler_Y.fit_transform(Y[nz].reshape(-1, 1)).flatten() 

# Scale Z with the same scaler as Y
z = scaler_Y.transform(Z)

# Split the data for training and validation as required
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.2, random_state=42)
x_ts, x_vl, y_ts, y_vl = train_test_split(x_ts, y_ts, test_size=0.01, random_state=42)

y_tr2, y_ts2, z_tr, z_ts = train_test_split(y, z, test_size=0.2, random_state=42)
y_ts2, y_vl, z_ts, z_vl = train_test_split(y_ts2, z_ts, test_size=0.01, random_state=42)

In [3]:
tuner_dir = 'trials'

if os.path.exists(tuner_dir):
    shutil.rmtree(tuner_dir)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
# Define the DNN model with hyperparameter tuning
class DNNHyperModel(kt.HyperModel):
    def build(self, hp):
        model = tf.keras.Sequential()
        
        # Input layer
        model.add(layers.Input(shape=(y_ncols,)))

        # Tune the number of hidden layers (renamed to 'num_layers')
        num_layers = hp.Int('num_layers', 1, 3)  # Between 1 and 3 hidden layers

        for i in range(num_layers):
            # Tune the number of units per hidden layer
            units = hp.Choice(f'units_{i}', [32, 64, 128])  # Choices: 32, 64, 128 units
            # Add hidden layer with fixed activation function (ReLU)
            model.add(layers.Dense(units, activation='relu'))
        
        # Output layer with fixed activation function (Linear)
        model.add(layers.Dense(1, activation='sigmoid'))

        model.compile(
            optimizer='adam',
            loss='mse',
            metrics=['mae']
        )
        
        return model

# Set up the tuner for a random search
tuner = kt.RandomSearch(
    DNNHyperModel(),  # Use the defined hypermodel
    objective='val_loss',
    max_trials=100,
    executions_per_trial=1,
    directory='trials',
)

# Set up early stopping
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1.0e-6, 
    patience=10
)

lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=10, 
    min_lr=1e-6
)

# Print the summary of search space
tuner.search_space_summary()

I0000 00:00:1730315383.703554 1574990 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1730315383.759849 1574990 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1730315383.760149 1574990 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1730315383.762234 1574990 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Search space summary
Default search space size: 2
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Choice)
{'default': 32, 'conditions': [], 'values': [32, 64, 128], 'ordered': True}


In [5]:
# Run search with variable batch sizes
tuner.search(
    y_tr2, z_tr,
    epochs=200,
    validation_data=(y_ts2, z_ts),
    batch_size=128,
    callbacks=[early_stop]
)

Trial 2 Complete [00h 00m 33s]
val_loss: 0.0003123053757008165

Best val_loss So Far: 0.0003123053757008165
Total elapsed time: 00h 01m 02s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
2                 |3                 |num_layers
32                |32                |units_0
128               |128               |units_1
64                |128               |units_2

Epoch 1/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.0494 - mae: 0.1787 - val_loss: 0.0041 - val_mae: 0.0509
Epoch 2/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0040 - mae: 0.0499 - val_loss: 0.0036 - val_mae: 0.0481
Epoch 3/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0035 - mae: 0.0470 - val_loss: 0.0033 - val_mae: 0.0461
Epoch 4/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0032 - mae: 0.0449 - val_loss: 0.0030 - val_mae: 0.0438
Epoch 5/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0030 - mae: 0.0435 - val_loss: 0.0026 - val_mae: 0.0413
Epoch 

KeyboardInterrupt: 

In [ ]:
tuner.results_summary(num_trials=1)

In [ ]:
# Retrieve the best model and evaluate
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.summary()

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', \
                                              min_delta=1.0e-6, patience=5)

# Optionally, retrain on the full dataset with optimal hyperparameters
training = best_model.fit(x_tr, y_tr, epochs=1000, validation_data=(x_ts, y_ts), \
                          batch_size=8, callbacks=[stop_early])

In [ ]:
loss = best_model.predict(x_vl)
print("Loss MSE= ", loss[0])

# Save the best model
best_model.save("best_model.keras")

In [ ]:
plt.semilogy(training.history['loss'], ls='--', color='red', label='train')
plt.semilogy(training.history['val_loss'], color='orange', label='test')
plt.grid()
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.title("Training History")
plt.legend()
plt.show()

In [ ]:
# Assume y_pr contains the normalized predictions and y_vl is the validation data
y_pr = best_model.predict(x_vl)

# Initialize arrays to store denormalized values
Y_pr = np.copy(y_pr)
Y_vl = np.copy(y_vl)

# Process each row individually
for i in range(y_pr.shape[0]):
    # Step 1: Identify the index of the first negative value in the row
    if np.any(y_pr[i] < 0):  # Check if there is any negative value
        i_nz = np.where(y_pr[i] < 0)[0][0]
        
        # Step 2: Set all values after the first negative value to zero
        y_pr[i, i_nz:] = 0

    # Step 3: Create a mask to identify non-zero values for denormalization
    nz_pr = y_pr[i] != 0
    nz_dt = y_vl[i] != 0

    # Step 4: Apply inverse_transform only on non-zero values
    if np.any(nz_pr):  # Check if there are any non-zero values
        Y_pr[i, nz_pr] = scaler_Y.inverse_transform(y_pr[i, nz_pr].reshape(-1, 1)).flatten()
    if np.any(nz_dt):  # Check if there are any non-zero values in the original validation data
        Y_vl[i, nz_dt] = scaler_Y.inverse_transform(y_vl[i, nz_dt].reshape(-1, 1)).flatten()

# Plot the results
m = np.linspace(0.2, 3.0, 100)
for i in range(22):
    plt.plot(Y_vl[i], m, label='Data', color='green', linestyle='--')
    plt.plot(Y_pr[i], m, label='Predicted', color='red')
    plt.xlabel("Radius")
    plt.ylabel("Mass")
    plt.legend()
    plt.show()


In [ ]:
def post_process_radius(y_pr):
    for i in range(len(y_pr)):
        # Get the current predicted radius vector for the i-th sample
        radii = y_pr[i]

        # Find the last non-zero radius index
        i_max = np.min(np.nonzero(radii))

        # Step 1: Set all values after the last non-zero value to zero
        radii[i_max + 1:] = 0

        # Step 2: Linearly interpolate oscillating values between zero and last non-zero
        for j in range(i_max - 1, 0, -1):
            if radii[j] < 0 or radii[j] > radii[j + 1]:
                # Linearly interpolate between j-1 and the last valid non-zero value
                radii[j] = (radii[j-1] + radii[j+1]) / 2

        # Update the radius vector in the prediction array
        y_pr[i] = radii

    return y_pr

# Assuming y_pr is the predicted radius output from your model
y_pr = model.predict(x_vl)
print(y_pr[3])
y_pr2 = post_process_radius(y_pr)
print(y_pr2[3])
